**Результаты so far:**
*  Воспроизведен бейзлайн RSG




# Data

In [1]:
%%capture
%%bash
# change url if you want to work with a different RSG dataset
wget -q --show-progress "https://russiansuperglue.com/tasks/download/RuCoS" -O temp.zip
unzip temp.zip -d data

# remove unnecessary directories and files
rm temp.zip
rm -r data/__MACOSX
rm -r sample_data/

In [2]:
# Load necessary code files and models from https://github.com/RussianNLP/RussianSuperGLUE 
# to recreate TfIdf baseline

%%capture
%%bash
# load tfidf pickle created by RSG team
wget -q --show-progress "https://russiansuperglue.com/tasks/tf_idf" -O temp.zip
unzip temp.zip -d data
rm temp.zip

# Make sure you donwload with the raw file link
# Keep the link relevant to your dataset
wget -q --show-progress "https://github.com/RussianNLP/RussianSuperGLUE/raw/master/tfidf_baseline/RuCoS.py" -O RuCoS.py

In [51]:
import pandas as pd
import json
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 80)

class JSONL_handler():
    """ opens a jsonl file and turns it into a necessary data structure """
    
    def __init__(self, path):
        self.path = path # path to jsonl file

    def to_pandas(self):
        """ get jsonl file content as a pandas DataFrame"""

        text_df = pd.DataFrame(columns=['text', 'entities'])
        questions_df = pd.DataFrame(columns=['text_id',
                                             'question', 'answers'])

        lines = self.yield_lines()

        for passage_id, line in enumerate(lines):
            text, entities, questions = self.split_text_and_questions(line)
            text_df = text_df.append({'text':text, 'entities': entities}, 
                           ignore_index=True)
            for i in range(len(questions)):
                questions_df = questions_df.append({'text_id': passage_id,
                                    'question': questions[i]['query'],
                                     'answers': questions[i]['answers']},
                                    ignore_index=True)
        return text_df, questions_df

    def yield_lines(self):
        """ yields json lines one by one """
        with open(self.path) as f:
            for line in f:
                yield json.loads(line)


    def split_text_and_questions(self, line):
        """ transforms a complex json object into a single row dataframe"""
        text = line['passage']['text']
        entities = line['passage']['entities']
        questions = line['qas']

        return text, entities, questions

In [52]:
train = JSONL_handler('data/RuCoS/train.jsonl')
text, questions = train.to_pandas()

# valid = JSONL_handler('data/RuCoS/val.jsonl')
# valid = valid.to_pandas()

In [55]:
text.shape, questions.shape

((72193, 2), (72193, 3))

In [56]:
text.head()

,text,entities
0,"Наблюдатели полагают, что подоплекой теракта в Домодедово является провал кавказской политики российского правительства, указывает немецкая печать. Немецкая печать продолжает комментировать теракт в Домодедово. Так, газета Süddeutsche Zeitung пишет:\nНу, конечно же, после взрыва в Домодедово вновь обнаруживается ""кавказский след"". Его обнаруживают почти всегда, когда в России взрывается бомба в метро, в поезде или на рынке. И неважно, что очевидцы нередко не в состоянии сказать, был ли преступник женщиной в чадре или же мужчиной, грозившим всех уничтожить.Среди жертв взрыва в Домодедово есть граждане Германии, Великобритании, Австрии. Ежедневно в Москве совершают посадку более 3 десятков самолетов из Германии.\n@highlight\nНемецкий менеджер аэропорта Домодедово отвергает обвинения Медведева\n@highlight\nТеракт в Домодедово: системные просчеты\n@highlight\nКомментарий: Провал российских спецслужб","[{'start': 47, 'end': 57}, {'start': 199, 'end': 209}, {'start': 223, 'end': 242}, {'start': 281, 'end': 291}, {'start': 371, 'end': 377}, {'start': 582, 'end': 592}, {'start': 607, 'end': 615}, {'start': 617, 'end': 631}, {'start': 633, 'end': 640}, {'start': 654, 'end': 660}, {'start': 709, 'end': 717}, {'start': 758, 'end': 768}, {'start': 789, 'end': 798}, {'start': 819, 'end': 829}]"
1,"О вторжении на Украину танковой колонны из РФ сообщил представитель СНБО Украины Лысенко. Москва опровергла эту информацию, назвав ее провокацией. На территорию восточной Украины из России вошла колонна из 32 танков, заявили власти в Киеве. Помимо этого, границу с Луганской областью пересекли 30 грузовиков с бойцами, 16 гаубиц и другая военная техника, сообщил представитель Совета национальной безопасности и обороны (СНБО) Украины Андрей Лысенко на брифинге в Киеве в пятницу, 7 ноября. По его словам, колонна движется в направлении города Красный луч. Переброска военного оборудования и российских наемников на линии фронта продолжается, резюмировал представитель СНБО.\n@highlight\nВ Германии за сутки выявлено более 100 новых заражений коронавирусом\n@highlight\nРыночные цены на нефть рухнули из-за провала переговоров ОПЕК+\n@highlight\nВ Италии за сутки произошел резкий скачок смертей от COVID-19","[{'start': 15, 'end': 22}, {'start': 43, 'end': 45}, {'start': 68, 'end': 72}, {'start': 73, 'end': 80}, {'start': 81, 'end': 88}, {'start': 90, 'end': 96}, {'start': 171, 'end': 178}, {'start': 182, 'end': 188}, {'start': 234, 'end': 239}, {'start': 265, 'end': 283}, {'start': 421, 'end': 425}, {'start': 427, 'end': 434}, {'start': 435, 'end': 449}, {'start': 464, 'end': 469}, {'start': 544, 'end': 555}, {'start': 669, 'end': 673}, {'start': 688, 'end': 696}, {'start': 823, 'end': 827}, {'start': 842, 'end': 848}]"
2,"Год назад Владимир Путин вновь стал президентом России. С тех пор протесты против его возвращения в Кремль поутихли. Но это внешнее спокойствие - обманчиво, считает Инго Маннтойфель. Настоящая стабильность возможна только при наличии правового государства и демократических институтов, которые способны поддерживать в равновесии различные общественные интересы. Ведь то, что может произойти в противном случае, не хочется даже себе представлять. @header Стабильность возможна только в правовом государстве Помня о русской традиции насильственных революционных переворотов, которые неизменно отбрасывали страну назад, хочется надеяться, что в Кремле осознают необходимость эволюционного развития и, следовательно, либеральных реформ.\n@highlight\nВ Германии за сутки выявлено более 100 новых заражений коронавирусом\n@highlight\nКомментарий: Россия накануне эпидемии - виноватые назначены заранее\n@highlight\nТуризм в эпоху коронавируса: куда поехать? И ехать ли вообще?","[{'start': 10, 'end': 24}, {'start': 48, 'end': 54}, {'start': 100, 'end': 106}, {'start': 165, 'end': 181}, {'start': 642, 'end': 648}, {'start': 746, 'end': 754}, {'start': 837, 'end': 843}]"
3,"Союз девяти ведущих технических университетов Германии TU9 не только 

In [57]:
questions.head()

,text_id,question,answers
0,0,"Кроме того, серьезным вызовом для России становится стремительно развивающийся Китай.Еще в понедельник @placeholder в рамках спора о системе противоракетной обороны пригрозил размещением дополнительных ракетных комплексов.","[{'start': 789, 'end': 798, 'text': 'Медведева'}]"
1,1,"Россия категорически опровергла сообщение @placeholder, назвав его провокацией.","[{'start': 81, 'end': 88, 'text': 'Лысенко'}, {'start': 435, 'end': 449, 'text': 'Андрей Лысенко'}]"
2,2,"@placeholder, руководитель отдела Восточной Европы и главный редактор русской редакции Deutsche Welle","[{'start': 165, 'end': 181, 'text': 'Инго Маннтойфель'}]"
3,3,"@placeholder позиционирует себя как союз независимых университетов, которые заинтересованы не столько в государственной поддержке, сколько в сотрудничестве с реальным сектором экономики, производством и бизнесом.","[{'start': 55, 'end': 58, 'text': 'TU9'}, {'start': 192, 'end': 195, 'text': 'TU9'}]"
4,4,Согласно указу президента @placeholder особые меры безопасности в местах проведения турнира действуют с 25 мая по 25 июля.,"[{'start': 546, 'end': 552, 'text': 'России'}]"


# RSG Baseline

In [3]:
%%capture
!pip install jsonlines

In [ ]:
import pickle
import codecs
import joblib
import RuCoS

vect = joblib.load("data/tfidf.pkl")

In [10]:
train_path = "data/RuCoS/train.jsonl"
val_path = "data/RuCoS/val.jsonl"
test_path = "data/RuCoS/test.jsonl"

_, RuCoS_scores = RuCoS.eval_RuCoS(train_path, val_path, test_path, vect)


val_em = RuCoS_scores['val'][0]
val_f1 = RuCoS_scores['val'][1]

print(f"EM Score on Validation: {val_em}") # should be around 0.22

print(f"F1 Score on Validation: {val_f1}") # should be around 0.23

EM Score on Validation: 0.22964233865646033
F1 Score on Validation: 0.235315208305838
